Lavoro sui dataset iniziali con tutte le etichette

In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import sys
sys.path.insert(1, '')
from utility import Dataset

train_df = pd.read_csv(f'KDDTrain+.txt', header=None)
test_df = pd.read_csv(f'KDDTest+.txt', header=None)

columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
           'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
           'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
           'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
           'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
           'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
           'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
           'dst_host_srv_rerror_rate', 'label', 'score']
train_df.columns = columns
test_df.columns = columns

nominal_features = ['protocol_type', 'service', 'flag']
binary_features = ['land', 'logged_in', 'root_shell', 'su_attempted', 'is_host_login', 'is_guest_login']
numeric_features = [feature for feature in columns if feature not in nominal_features + binary_features + ['label', 'score']]

# 'su_attempted' dovrebbe essere binario ma ha 3 valori. Sostituiamo '2.0' con '0.0'
train_df['su_attempted'] = train_df['su_attempted'].replace(2, 0)
test_df['su_attempted'] = test_df['su_attempted'].replace(2, 0)

# 'num_outbound_cmds' ha sempre lo stesso valore quindi possiamo dropparla
train_df = train_df.drop('num_outbound_cmds', axis=1)
test_df = test_df.drop('num_outbound_cmds', axis=1)

** remove highly uncorrelated features ?? **

# One Hot Encoding for categorical variables
trovare una motivazione al perchè ho scelto questa codifica

In [40]:
enc = OneHotEncoder()

train_encoded_array = enc.fit_transform(train_df[nominal_features]).toarray()
test_encoded_array = enc.transform(test_df[nominal_features]).toarray()

new_columns = []
for i, feature in enumerate(nominal_features):
    new_columns.extend([f"{feature}_{str(cat)}" for cat in enc.categories_[i]])

train_ohe = train_df.drop(nominal_features, axis=1)
train_ohe = pd.concat([train_df, pd.DataFrame(train_encoded_array, columns=new_columns)], axis=1)

test_ohe = test_df.drop(nominal_features, axis=1)
test_ohe = pd.concat([test_df, pd.DataFrame(test_encoded_array, columns=new_columns)], axis=1)

print(train_ohe)
print(test_ohe)

        duration protocol_type   service flag  src_bytes  dst_bytes  land  \
0              0           tcp  ftp_data   SF        491          0     0   
1              0           udp     other   SF        146          0     0   
2              0           tcp   private   S0          0          0     0   
3              0           tcp      http   SF        232       8153     0   
4              0           tcp      http   SF        199        420     0   
...          ...           ...       ...  ...        ...        ...   ...   
125968         0           tcp   private   S0          0          0     0   
125969         8           udp   private   SF        105        145     0   
125970         0           tcp      smtp   SF       2231        384     0   
125971         0           tcp    klogin   S0          0          0     0   
125972         0           tcp  ftp_data   SF        151          0     0   

        wrong_fragment  urgent  hot  num_failed_logins  logged_in  \
0     